# PLRデータ分析例 - AWARE Frameworkデータ

PLRに保存したデータの取得例です。

本環境は [Jupyter Notebook](https://jupyter.org/) の形で構成されており、[Pythonスクリプト](https://www.python.jp/)を用いて記述しています。

ライブラリは公式イメージ [jupyter/scipy-notebook](https://jupyter-docker-stacks.readthedocs.io/en/latest/using/selecting.html#jupyter-scipy-notebook) のものがインストールされています。

# このデータ分析例が想定するタスク

このデータ分析例は、AWARE Frameworkから得られたデータのうち、**被験者本人のPersonaryに格納されるデータ** の分析方法を対象としています。

https://goemon.cloud/t/9MsyRIPkGmgexbE4809l

AWARE Framework用のQRコードが開かない場合は、リロードして再度試してみてください。

# チャンネルの取得

処理したいデータが格納されたチャンネルを取得します。GO-E-MONにより格納されたデータを処理したい場合は、 `cog-pds-log` となります。


> Jupyter Notebook Serverの起動直後など、実行がなかなか終わらない状態になる可能性があります。
> 実行状態が `[*]` のまま先に進まない場合は 中断(■) を押して一旦中断し、再度実行してみてください。

In [1]:
import asyncio
from plrfs.rpc_client import PLRFSClient

# クライアントライブラリを初期化
loop = asyncio.get_event_loop()

client = await PLRFSClient(loop).connect()

# チャンネル一覧を取得
channels = await client.get_files([])
for ch in channels:
    print('Channel', repr(ch['name']))

Channel 'family'
Channel 'coworkers'
Channel 'friends'
Channel 'ログ'
Channel 'ログスキーマ'
Channel 'テスト１'
Channel 'cog-pds-log'
Channel 'テスト(from googleDrive:yazawa@voice-research.com)'


参照したいチャンネル名を以下に記述します。

In [2]:
channel_name = 'cog-pds-log'
channel_name

'cog-pds-log'

In [3]:
channels = await client.get_files([])
cogtask_me_chs = [c for c in channels if c['name'] == channel_name]
assert len(cogtask_me_chs) > 0, 'cogtask.meのチャンネルが見つかりません。'
cogtask_me_ch = cogtask_me_chs[0]
cogtask_me_ch

{'kind': 'folder', 'id': '20211102075548_UKqR', 'name': 'cog-pds-log'}

データの読み込みは `PLRFSClient`から実施できます。これは以下のような関数を持っています。

In [4]:
help(client)

Help on PLRFSClient in module plrfs.rpc_client object:

class PLRFSClient(builtins.object)
 |  PLRFSClient(loop)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, loop)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  async connect(self, host='tcp://localhost:5555')
 |  
 |  async get_file(self, path)
 |  
 |  async get_files(self, path)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)



ファイルシステムを模した構造になっていて、 `client.get_files` で指定したチャンネル、アイテムの配下のアイテム一覧を取得することができます。また、`client.get_file` で指定したチャンネル、アイテムの配下のデータを取得することができます。


# AWARE Frameworkデータの取得

試しに、チャンネルからAWARE Frameworkで得られたデータを取り出してみましょう。

In [5]:
import re

# 正規表現で、処理対象とするアイテムの名前のパターンを定義する
item_pattern = re.compile(r'^AWARE Frameworkデータが同期されました$')
item_pattern

re.compile(r'^AWARE Frameworkデータが同期されました$', re.UNICODE)

In [9]:
import json
import pandas as pd

timeline_item_data = []
# チャンネル内のアイテムを取得する
timeline_items = await client.get_files([cogtask_me_ch['id']])

for item in timeline_items:
    print('Processing...', item['id'])
    # アイテムのプロパティを取得する
    timeline_properties = await client.get_files([cogtask_me_ch['id'], item['id']])
    if 'cogPDSJSON' not in [p['name'] for p in timeline_properties]:
        continue
    assert 'cnt' in [p['name'] for p in timeline_properties]
    summary = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cnt'][0]])
    summary_s = summary['content'].decode('utf8')
    if not item_pattern.match(summary_s):
        continue
    
    assert 'begin' in [p['name'] for p in timeline_properties]
    begin = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'begin'][0]])
    begin_s = begin['content'].decode('utf8')
    
    if 'cogPDSUser' in [p['name'] for p in timeline_properties]:
        cogPDSUser = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cogPDSUser'][0]])
    else:
        cogPDSUser = None

    user_s = cogPDSUser['content'].decode('utf8') if cogPDSUser is not None else None
    
    cogPDSJSON = await client.get_file([cogtask_me_ch['id'], item['id'], [p['id'] for p in timeline_properties if p['name'] == 'cogPDSJSON'][0]])
    data = json.loads(cogPDSJSON['content'])

    if 'category' not in data:
        print('WARNING: category not defined:', begin_s)
        continue

    assert 'data' in data
    for elem in json.loads(data['data']):
        # dataにはリストが格納されるので、これを展開する
        timeline_item_data.append((summary_s, begin_s, user_s, data['category'], elem))

df = pd.DataFrame(timeline_item_data, columns=['Summary', 'Time', 'User', 'Category', 'Detail'])
df

Processing... #20211102075628_QYFv
Processing... #20211104051707_YkQs
Processing... #20211216033506_4xjl
Processing... #20211216033906_4oU4
Processing... #20211218022807_7VbR
Processing... #20211218023232_BLUW
Processing... #20211228015806_xVhM
Processing... #20220318044510_gbpQ
Processing... #20220402090436_0PiI
Processing... #20220402092905_1Hgv
Processing... #20220402093206_rILv
Processing... #20220407125222_fGQ7
Processing... #20220407125217_tcA0
Processing... #20220407125212_b6Tc
Processing... #20220407125226_8uLs
Processing... #20220407125207_HYZs
Processing... #20220407125306_vv18
Processing... #20220408050838_3mwf
Processing... #20220408050906_9r0U
Processing... #20220418133508_fJO6
Processing... #20221006101307_EJfS
Processing... #20221006101507_fQ-N
Processing... #20221006101511_rKTE
Processing... #20221006101807_t7Wi
Processing... #20221006101818_xF9l
Processing... #20221006102107_I7IM
Processing... #20221006102112_FWLU
Processing... #20221006102406_vNof
Processing... #20221

,Summary,Time,User,Category,Detail
0,AWARE Frameworkデータが同期されました,2024-04-29T12:56:07.384Z,9IQe87obCjTC7mUhuSnj,battery,"{'battery_adaptor': 0, 'battery_health': 0, 'b..."
1,AWARE Frameworkデータが同期されました,2024-04-29T12:56:07.384Z,9IQe87obCjTC7mUhuSnj,battery,"{'battery_adaptor': 0, 'battery_health': 0, 'b..."
2,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.040Z,9IQe87obCjTC7mUhuSnj,ios_aware_log,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
3,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.040Z,9IQe87obCjTC7mUhuSnj,ios_aware_log,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
4,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.040Z,9IQe87obCjTC7mUhuSnj,ios_aware_log,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
...,...,...,...,...,...
374,AWARE Frameworkデータが同期されました,2024-04-29T21:32:53.782Z,4CsZ0rrqHCupm1rmres5,ios_aware_log,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
375,AWARE Frameworkデータが同期されました,2024-04-29T21:32:53.782Z,4CsZ0rrqHCupm1rmres5,ios_aware_log,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
376,AWARE Frameworkデータが同期されました,2024-04-29T21:32:53.782Z,4CsZ0rrqHCupm1rmres5,ios_aware_log,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
377,AWARE Frameworkデータが同期されました,2024-04-29T21:32:53.782Z,4CsZ0rrqHCupm1rmres5,ios_aware_log,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...


dfにはAWARE Frameworkで得られたデータが全て格納されます。

デバイス使用時間(`plugin_device_usage`)のみを参照するには以下のようにセルを実行します。

In [10]:
df[df['Category'] == 'plugin_device_usage']

,Summary,Time,User,Category,Detail
48,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.219Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
49,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.219Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
56,AWARE Frameworkデータが同期されました,2024-04-29T13:03:01.039Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
57,AWARE Frameworkデータが同期されました,2024-04-29T13:03:01.039Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
93,AWARE Frameworkデータが同期されました,2024-04-29T13:38:52.475Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
94,AWARE Frameworkデータが同期されました,2024-04-29T13:38:52.475Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
173,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
174,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
175,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...
176,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,{'device_id': 'f161f878-da5b-48df-9636-72fd8e1...


このようにして、Personary上のデータをメモリに読み込むことができます。これらの内容を**暗号化等をかけていないファイル等に出力する際は十分に取り扱いに注意**してください。

各データの `Detail` にはログが記録されています。

In [11]:
df[df['Category'] == 'plugin_device_usage']['Detail'].values[-1]

{'device_id': 'f161f878-da5b-48df-9636-72fd8e1abf64',
 'elapsed_device_off': 730059.9749088287,
 'elapsed_device_on': 0,
 'timestamp': 1714426278605}

デバイス使用時間を処理する場合は、Detailの構造を展開します。

コードの記述はChatGPTに聞いても良いかもしれません。 https://chat.openai.com/share/a0575d45-50f3-442e-bd87-e980ca0daf6f

In [15]:
# 対象はplugin_device_usageとする
df_ = df[df['Category'] == 'plugin_device_usage']

# Detailカラムの辞書をカラムに展開
detail_expanded = df_['Detail'].apply(pd.Series)

# 元のDataFrameに展開したカラムを結合
df_expanded = pd.concat([df_.drop('Detail', axis=1), detail_expanded], axis=1)

df_expanded

,Summary,Time,User,Category,device_id,elapsed_device_off,elapsed_device_on,timestamp
48,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.219Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,6.342468e+06,0.000000,1714393079925
49,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.219Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,2.275273e+06,0.000000,1714395355197
56,AWARE Frameworkデータが同期されました,2024-04-29T13:03:01.039Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,319177.801132,1714395674375
57,AWARE Frameworkデータが同期されました,2024-04-29T13:03:01.039Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,8.672460e+04,0.000000,1714395761100
93,AWARE Frameworkデータが同期されました,2024-04-29T13:38:52.475Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,338284.371853,1714396099384
94,AWARE Frameworkデータが同期されました,2024-04-29T13:38:52.475Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,1.830817e+06,0.000000,1714397930201
173,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,1.576361e+07,0.000000,1714421882051
174,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,224249.723911,1714422106302
175,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,3.541968e+05,0.000000,1714422460499
176,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,9328.276873,1714422469827


GO-E-MONからPersonaryへのデータ格納は、ネットワーク状況等によって、複数重複して格納される場合がありますので、重複排除しておきます。
また、timestampはepoch time https://en.wikipedia.org/wiki/Unix_time (ミリ秒)で格納されますので、Pandasで扱いやすいよう、datetimeに変換しておきます。

In [18]:
df_expanded = df_expanded.drop_duplicates(subset=['timestamp'])

# エポックタイムを日時に変換（初期設定はUTC）
df_expanded['datetime'] = pd.to_datetime(df_expanded['timestamp'] / 1000, unit='s')

# UTCからJST（日本標準時、UTC+9）に変換
df_expanded['datetime'] = df_expanded['datetime'].dt.tz_localize('UTC').dt.tz_convert('Asia/Tokyo')

df_expanded

,Summary,Time,User,Category,device_id,elapsed_device_off,elapsed_device_on,timestamp,datetime
48,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.219Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,6.342468e+06,0.000000,1714393079925,2024-04-29 21:17:59.924999936+09:00
49,AWARE Frameworkデータが同期されました,2024-04-29T12:56:10.219Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,2.275273e+06,0.000000,1714395355197,2024-04-29 21:55:55.196999936+09:00
56,AWARE Frameworkデータが同期されました,2024-04-29T13:03:01.039Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,319177.801132,1714395674375,2024-04-29 22:01:14.375000064+09:00
57,AWARE Frameworkデータが同期されました,2024-04-29T13:03:01.039Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,8.672460e+04,0.000000,1714395761100,2024-04-29 22:02:41.100000+09:00
93,AWARE Frameworkデータが同期されました,2024-04-29T13:38:52.475Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,338284.371853,1714396099384,2024-04-29 22:08:19.384000+09:00
94,AWARE Frameworkデータが同期されました,2024-04-29T13:38:52.475Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,1.830817e+06,0.000000,1714397930201,2024-04-29 22:38:50.200999936+09:00
173,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,1.576361e+07,0.000000,1714421882051,2024-04-30 05:18:02.051000064+09:00
174,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,224249.723911,1714422106302,2024-04-30 05:21:46.302000128+09:00
175,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,3.541968e+05,0.000000,1714422460499,2024-04-30 05:27:40.499000064+09:00
176,AWARE Frameworkデータが同期されました,2024-04-29T20:27:51.185Z,9IQe87obCjTC7mUhuSnj,plugin_device_usage,f161f878-da5b-48df-9636-72fd8e1abf64,0.000000e+00,9328.276873,1714422469827,2024-04-30 05:27:49.826999808+09:00


それぞれの値の説明は https://awareframework.com/ も参考にしてください。